In [15]:
from tensorflow import keras
from keras import layers
from random import randint

In [56]:
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    '../data/pngimages',
    subset='both',
    seed=randint(0, 2**32),
    validation_split=0.2,
    image_size=(128, 128),
    color_mode='rgb'
)


Found 1125 files belonging to 15 classes.
Using 900 files for training.
Using 225 files for validation.


In [53]:
## Preferably put this in a different module..
def build_model():
    model = keras.Sequential([
        # layers.Resizing(128, 128), # This is already handled by the loader
        layers.RandomFlip('horizontal'),
        layers.Rescaling(1 / 255),
        layers.Conv2D(8, 3, input_shape=(128, 128, 3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(15, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model

In [54]:
model = build_model()
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(
    train_ds,
    epochs=100,
    batch_size=32,
    validation_data=val_ds,
    callbacks=[callback]
)


Epoch 1/100
29/29 [==============================] - 14s 361ms/step - loss: 3.4322 - accuracy: 0.1789 - val_loss: 2.0466 - val_accuracy: 0.3556
Epoch 2/100
29/29 [==============================] - 13s 349ms/step - loss: 1.7671 - accuracy: 0.4356 - val_loss: 1.4816 - val_accuracy: 0.5467
Epoch 3/100
29/29 [==============================] - 13s 339ms/step - loss: 1.2296 - accuracy: 0.6322 - val_loss: 1.2536 - val_accuracy: 0.5956
Epoch 4/100
29/29 [==============================] - 13s 336ms/step - loss: 1.0403 - accuracy: 0.6911 - val_loss: 0.9208 - val_accuracy: 0.6933
Epoch 5/100
29/29 [==============================] - 13s 340ms/step - loss: 0.8047 - accuracy: 0.7556 - val_loss: 0.7298 - val_accuracy: 0.7600
Epoch 6/100
29/29 [==============================] - 13s 343ms/step - loss: 0.6635 - accuracy: 0.7900 - val_loss: 0.8052 - val_accuracy: 0.7289
Epoch 7/100
29/29 [==============================] - 13s 337ms/step - loss: 0.5775 - accuracy: 0.8044 - val_loss: 0.8432 - val_accuracy: